In [1]:
import json
import math
from pprint import pprint
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as pltfrom
from sklearn.metrics.pairwise import cosine_similarity

from keras.layers import Embedding, LSTM, Dense, Conv1D, MaxPooling1D, Dropout, Activation
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

 
import numpy as np
np.seterr(divide='ignore', invalid='ignore')

Using TensorFlow backend.


{'divide': 'warn', 'invalid': 'warn', 'over': 'warn', 'under': 'ignore'}

In [2]:
with open("Headline_Trainingdata.json") as f:
    reviews_train = json.load(f)

with open("Headline_Trialdata.json") as f:
    reviews_trial = json.load(f)

with open("Headlines_Testdata.json", encoding="utf8") as f:
    reviews_test = json.load(f)

maxlen = 0
#pprint(reviews_train)
#X_train = [review['title'] for review in reviews_train]
#X_trial = [review['title'] for review in reviews_trial]
#Y_train = [0 if review['sentiment']<=0 else 1 for review in reviews_train]
#Y_trial = [0 if review['sentiment']<=0 else 1 for review in reviews_trial]

X_train = []
X_test = [review['title'] for review in reviews_test]
output = []
Y_train = []
Y_test = []
for review in reviews_train :
    X_train.append(review['title'])
    Y_train.append(0 if review['sentiment']<0 else 1)
    output.append(review['sentiment'])
    if maxlen < len(review['title'].split()) :
        maxlen = len(review['title'].split())
    
for review in reviews_trial :
    reviews_train.append(review)
    X_train.append(review['title'])
    Y_train.append(0 if review['sentiment']<0 else 1)
    output.append(review['sentiment'])
    if maxlen < len(review['title'].split()) :
        maxlen = len(review['title'].split())

testMax = 0
for review in reviews_test :
    if testMax < len(review['title'].split()) :
        testMax = len(review['title'].split())
print(len(X_train)+len(X_test), len(Y_train))
print(testMax)


1647 1156
18


In [3]:
tokenizer = Tokenizer(num_words=15000)
tokenizer.fit_on_texts(X_train)
sequences = tokenizer.texts_to_sequences(X_train)
data = pad_sequences(sequences, maxlen=50)

In [4]:
model = Sequential()
model.add(Embedding(15000, 16, input_length=50))
model.add(LSTM(16, dropout=0.15, recurrent_dropout=0.15))
#model.add(Dense(4, activation='tanh'))
model.add(Dense(1, activation='tanh'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 16)            240000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 16)                2112      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 242,129
Trainable params: 242,129
Non-trainable params: 0
_________________________________________________________________


In [5]:


history = model.fit(data, np.array(Y_train), validation_split=0.4, epochs=75,verbose = 1)

#plot_training_curves(history.history);

Train on 693 samples, validate on 463 samples
Epoch 1/75
693/693 [==============================] - 2s - loss: 2.2094 - acc: 0.3867 - val_loss: 1.3071 - val_acc: 0.4147
Epoch 2/75
693/693 [==============================] - 1s - loss: 1.0932 - acc: 0.3867 - val_loss: 0.7971 - val_acc: 0.4147
Epoch 3/75
693/693 [==============================] - 1s - loss: 0.6984 - acc: 0.5570 - val_loss: 0.6661 - val_acc: 0.5940
Epoch 4/75
693/693 [==============================] - 1s - loss: 0.6605 - acc: 0.5974 - val_loss: 0.6569 - val_acc: 0.5918
Epoch 5/75
693/693 [==============================] - 1s - loss: 0.6008 - acc: 0.6854 - val_loss: 0.6439 - val_acc: 0.6544
Epoch 6/75
693/693 [==============================] - 1s - loss: 0.5402 - acc: 0.7576 - val_loss: 0.6259 - val_acc: 0.6609
Epoch 7/75
693/693 [==============================] - 1s - loss: 0.4698 - acc: 0.8499 - val_loss: 0.6303 - val_acc: 0.6782
Epoch 8/75
693/693 [==============================] - 1s - loss: 0.3765 - acc: 0.9019 - val_l

693/693 [==============================] - 1s - loss: 0.0195 - acc: 0.8874 - val_loss: 1.5738 - val_acc: 0.6782
Epoch 67/75
693/693 [==============================] - 1s - loss: 0.0178 - acc: 0.9048 - val_loss: 1.5924 - val_acc: 0.6695
Epoch 68/75
693/693 [==============================] - 1s - loss: 0.0189 - acc: 0.8990 - val_loss: 1.5825 - val_acc: 0.6933
Epoch 69/75
693/693 [==============================] - 1s - loss: 0.0200 - acc: 0.8932 - val_loss: 1.5923 - val_acc: 0.6868
Epoch 70/75
693/693 [==============================] - 1s - loss: 0.0197 - acc: 0.8860 - val_loss: 1.5769 - val_acc: 0.6717
Epoch 71/75
693/693 [==============================] - 1s - loss: 0.0219 - acc: 0.8918 - val_loss: 1.6166 - val_acc: 0.6847
Epoch 72/75
693/693 [==============================] - 1s - loss: 0.0180 - acc: 0.8961 - val_loss: 1.5722 - val_acc: 0.6890
Epoch 73/75
693/693 [==============================] - 1s - loss: 0.0194 - acc: 0.8846 - val_loss: 1.6088 - val_acc: 0.6868
Epoch 74/75
693/693 

In [6]:
sequences = tokenizer.texts_to_sequences(np.array(X_train))
data = pad_sequences(sequences, maxlen=50)
predictions = model.predict(data)
predictedVals = [pr[0] for pr in predictions ]

similarity = cosine_similarity(output,predictedVals) 


print(similarity)

#print("\t\t\tHEADLINE\t\t\t\t| GS | PS |")
#for i in range(len(predictions)):
#    print(X_train[i],"|",output[i],"|","%.3f" % predictions[i],"|")

[[ 0.5390725]]


C:\Users\Anuja\Anaconda3\lib\site-packages\sklearn\utils\validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
C:\Users\Anuja\Anaconda3\lib\site-packages\sklearn\utils\validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


In [7]:
df = pd.DataFrame.from_dict(reviews_train)
df['Predicted Sentiment Score'] = predictedVals

df.head()

,company,id,sentiment,title,Predicted Sentiment Score
0,Morrisons,2,0.430,Morrisons book second consecutive quarter of s...,0.999733
1,IMI,3,-0.344,IMI posts drop in first-quarter organic revenu...,-0.290707
2,Glencore,4,0.340,Glencore to refinance its short-term debt earl...,0.999891
3,Ryanair,5,0.259,EasyJet attracts more passengers in June but s...,0.999877
4,Barclays,6,-0.231,Barclays 'bad bank' chief to step down,-0.490590


In [8]:
df.to_csv('Task2_train.csv', sep = ",",encoding="utf-8")